In [10]:
import cv2
import numpy as np

# Get a VideoCapture object from video and store it in vs
vc = cv2.VideoCapture("April_09_brush_hair_u_nm_np1_ba_goo_0.avi")
# Read first frame
ret, first_frame = vc.read()
# Scale and resize image
resize_dim = 600
max_dim = max(first_frame.shape)
scale = resize_dim/max_dim
first_frame = cv2.resize(first_frame, None, fx=scale, fy=scale)
# Convert to gray scale 
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)


# Create mask
mask = np.zeros_like(first_frame)
# Sets image saturation to maximum
mask[..., 1] = 255


out = cv2.VideoWriter('video.avi',-1,1,(600, 600))

while(vc.isOpened()):
    # Read a frame from video
    ret, frame = vc.read()
    cv2.imshow('porcoddio', frame)
    
    # Convert new frame format`s to gray scale and resize gray frame obtained
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, None, fx=scale, fy=scale)

    # Calculate dense optical flow by Farneback method
    # https://docs.opencv.org/3.0-beta/modules/video/doc/motion_analysis_and_object_tracking.html#calcopticalflowfarneback
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, pyr_scale = 0.5, levels = 5, winsize = 11, iterations = 5, poly_n = 5, poly_sigma = 1.1, flags = 0)
    # Compute the magnitude and angle of the 2D vectors
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    # Set image hue according to the optical flow direction
    mask[..., 0] = angle * 180 / np.pi / 2
    # Set image value according to the optical flow magnitude (normalized)
    mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    # Convert HSV to RGB (BGR) color representation
    cv2.imshow('porcoddio1', mask)
    rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)

    # Resize frame size to match dimensions
    frame = cv2.resize(frame, None, fx=scale, fy=scale)
    
    # Open a new window and displays the output frame
    dense_flow = cv2.addWeighted(frame, 1,rgb, 2, 0)
    cv2.imshow("Dense optical flow", dense_flow)
    out.write(dense_flow)
    # Update previous frame
    prev_gray = gray
    # Frame are read by intervals of 1 millisecond. The programs breaks out of the while loop when the user presses the 'q' key
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

KeyboardInterrupt: 

In [1]:
import numpy as np
import cv2 as cv
cap = cv.VideoCapture(cv.samples.findFile("vtest.avi"))
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255
while(1):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    cv.imshow('frame2', bgr)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv.imwrite('opticalfb.png', frame2)
        cv.imwrite('opticalhsv.png', bgr)
    prvs = next
cv.destroyAllWindows()

# Test 2

In [2]:
import cv2
import numpy as np


def dense_optical_flow(method, video_path, params=[], to_gray=False):
    # read the video
    cap = cv2.VideoCapture(video_path)
    # Read the first frame
    ret, old_frame = cap.read()

    # crate HSV & make Value a constant
    hsv = np.zeros_like(old_frame)
    hsv[..., 1] = 255

    # Preprocessing for exact method
    if to_gray:
        old_frame = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

    while True:
        # Read the next frame
        ret, new_frame = cap.read()
        frame_copy = new_frame
        if not ret:
            break
        # Preprocessing for exact method
        if to_gray:
            new_frame = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)
        # Calculate Optical Flow
        flow = method(old_frame, new_frame, None, *params)

        # Encoding: convert the algorithm's output into Polar coordinates
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        # Use Hue and Saturation to encode the Optical Flow
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        # Convert HSV image into BGR for demo
        bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        cv2.imshow("frame", frame_copy)
        cv2.imshow("optical flow", bgr)
        k = cv2.waitKey(25) & 0xFF
        if k == 27:
            break
        old_frame = new_frame

In [3]:
def lucas_kanade_method(video_path):
    cap = cv2.VideoCapture(video_path)
    # params for ShiTomasi corner detection
    feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
    # Parameters for lucas kanade optical flow
    lk_params = dict(
        winSize=(15, 15),
        maxLevel=2,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03),
    )
    # Create some random colors
    color = np.random.randint(0, 255, (100, 3))
    # Take first frame and find corners in it
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
    # Create a mask image for drawing purposes
    mask = np.zeros_like(old_frame)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(
            old_gray, frame_gray, p0, None, **lk_params
        )
        # Select good points
        good_new = p1[st == 1]
        good_old = p0[st == 1]
        # draw the tracks
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), 2)
            frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1)
        img = cv2.add(frame, mask)
        cv2.imshow("frame", img)
        k = cv2.waitKey(25) & 0xFF
        if k == 27:
            break
        if k == ord("c"):
            mask = np.zeros_like(old_frame)
        # Now update the previous frame and previous points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

In [27]:
# "lucaskanade":
video_path = "Increase_stride_length_by_running_stairs__working_step_over_action_speed_development_climb_stairs_f_cm_np1_le_med_1.avi"
lucas_kanade_method(video_path)
cv2.destroyAllWindows()

video_path = "Stabilized/Increase_stride_length_by_running_stairs__working_step_over_action_speed_development_climb_stairs_f_cm_np1_le_med_1.avi"
lucas_kanade_method(video_path)
cv2.destroyAllWindows()

In [25]:
# lucaskanade_dense
method = cv2.optflow.calcOpticalFlowSparseToDense
dense_optical_flow(method, "Increase_stride_length_by_running_stairs__working_step_over_action_speed_development_climb_stairs_f_cm_np1_le_med_1.avi", to_gray=True)
cv2.destroyAllWindows()

method = cv2.optflow.calcOpticalFlowSparseToDense
dense_optical_flow(method, "Stabilized/Increase_stride_length_by_running_stairs__working_step_over_action_speed_development_climb_stairs_f_cm_np1_le_med_1.avi", to_gray=True)
cv2.destroyAllWindows()

In [29]:
video_path = "Increase_stride_length_by_running_stairs__working_step_over_action_speed_development_climb_stairs_f_cm_np1_le_med_1.avi"
method = cv2.calcOpticalFlowFarneback
params = [0.5, 3, 15, 3, 5, 1.2, 0]  # Farneback's algorithm parameters
dense_optical_flow(method, video_path, params, to_gray=True)
cv2.destroyAllWindows()

video_path = "Stabilized/Increase_stride_length_by_running_stairs__working_step_over_action_speed_development_climb_stairs_f_cm_np1_le_med_1.avi"
method = cv2.calcOpticalFlowFarneback
params = [0.5, 3, 15, 3, 5, 1.2, 0]  # Farneback's algorithm parameters
dense_optical_flow(method, video_path, params, to_gray=True)
cv2.destroyAllWindows()


In [28]:
#RLOF
video_path = "Increase_stride_length_by_running_stairs__working_step_over_action_speed_development_climb_stairs_f_cm_np1_le_med_1.avi"
method = cv2.optflow.calcOpticalFlowDenseRLOF
dense_optical_flow(method, video_path)
cv2.destroyAllWindows()

video_path = "Stabilized/Increase_stride_length_by_running_stairs__working_step_over_action_speed_development_climb_stairs_f_cm_np1_le_med_1.avi"
method = cv2.optflow.calcOpticalFlowDenseRLOF
dense_optical_flow(method, video_path)
cv2.destroyAllWindows()



In [16]:
cv2.destroyAllWindows()


In [5]:
help(cv2.optflow)

Help on module cv2.optflow in cv2:

NAME
    cv2.optflow

FUNCTIONS
    DenseRLOFOpticalFlow_create(...)
        DenseRLOFOpticalFlow_create([, rlofParam[, forwardBackwardThreshold[, gridStep[, interp_type[, epicK[, epicSigma[, epicLambda[, ricSPSize[, ricSLICType[, use_post_proc[, fgsLambda[, fgsSigma[, use_variational_refinement]]]]]]]]]]]]]) -> retval
        .   *    @param rlofParam see optflow::RLOFOpticalFlowParameter
        .        *    @param forwardBackwardThreshold see setForwardBackward
        .        *    @param gridStep see setGridStep
        .        *    @param interp_type see setInterpolation
        .        *    @param epicK see setEPICK
        .        *    @param epicSigma see setEPICSigma
        .        *    @param epicLambda see setEPICLambda
        .        *    @param ricSPSize see setRICSPSize
        .        *    @param ricSLICType see setRICSLICType
        .        *    @param use_post_proc see setUsePostProc
        .        *    @param fgsLambda

In [1]:
import cv2
cv2.__version__
cv2.cuda.getCudaEnabledDeviceCount()

1

In [ ]:
def main():
    #parser = ArgumentParser()
    #parser.add_argument(
    #    "--algorithm",
    #    choices=["farneback", "lucaskanade", "lucaskanade_dense", "rlof"],
    #    required=True,
    #    help="Optical flow algorithm to use",
    #)
    #parser.add_argument(
    #    "--video_path", default="videos/cat.mp4", help="Path to the video",
    #)

    video_path = "videos/people.mp4"
    algorithm = "lucaskanade_dense"
    
    if algorithm == "lucaskanade":
        lucas_kanade_method(video_path)
    elif algorithm == "lucaskanade_dense":
        method = cv2.optflow.calcOpticalFlowSparseToDense
        dense_optical_flow(method, video_path, to_gray=True)
    elif algorithm == "farneback":
        method = cv2.calcOpticalFlowFarneback
        params = [0.5, 3, 15, 3, 5, 1.2, 0]  # Farneback's algorithm parameters
        dense_optical_flow(method, video_path, params, to_gray=True)
    elif algorithm == "rlof":
        method = cv2.optflow.calcOpticalFlowDenseRLOF
        dense_optical_flow(method, video_path)

if __name__ == "__main__":
    main()